In [ ]:
import ee, geetools

ee.Initialize()

In [ ]:
fc = (
    ee.FeatureCollection("FAO/GAUL/2015/level0")
    .filter(ee.Filter.inList("ADM0_CODE", [122, 237, 85]))
)
geom = fc.geetools.mergeGeometries()

In [ ]:
import geemap
m = geemap.Map()
m.addLayer(geom, {}, "Countries")
m

In [ ]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', 110))
fc

In [ ]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(vatican).first()
image = image.select(["B4", "B2"]).geetools.interpolateBands([0, 3000], [0, 30])
values = image.reduceRegion(ee.Reducer.mean(), vatican, 1)
print(values.getInfo())

In [ ]:
image.getInfo()

In [ ]:
values = image.reduceRegion(ee.Reducer.first(), vatican, 1)

In [ ]:
values.getInfo()

In [4]:
import ee, geetools
from itertools import permutations

list([i for i in permutations([0,1], 2)])

[(0, 1), (1, 0)]

In [18]:
point0 = ee.Geometry.Point([0,0])
point1 = ee.Geometry.Point([0,1])
poly0 = point0.buffer(1)
poly1 = point1.buffer(1).bounds()
line = ee.Geometry.LineString([point1, point0], geodesic=True)
line.getInfo()
multiPoly = ee.Geometry.MultiPolygon([poly0, poly1], geodesic=True)
multiPoly.getInfo()
geometryCol = ee.Algorithms.GeometryConstructors.MultiGeometry([poly0, poly1, line, point0], crs="EPSG:4326", geodesic=True, maxError=1)

geometryCol.getInfo()

EEException: GeometryConstructors.MultiPolygon: Geometry coordinates do not have the same geodesic state. Specify an explicit geodesic state to reproject inputs.

In [6]:
new_fc = ee.FeatureCollection([poly1, multiPoly, geometryCol]).geetools.toPolygons()
new_fc.getInfo()

EEException: Projection: Argument 'crs': Invalid type.
Expected type: String.
Actual type: Boolean.
Actual value: true